In [1]:
import os
import skimage.io
import matplotlib.pyplot as plt

%matplotlib inline

os.chdir('../') # OPTIMIZATION EDIT: import most recent mrcnn code from local code

# Import Mask RCNN
from mrcnn.config import Config
import mrcnn.model as modellib
from mrcnn import utils

os.chdir('project')  # OPTIMIZATION EDIT: revert to current directory

# Root directory of the project
ROOT_DIR = os.path.abspath("../")
print('Project Directory: {}'.format(ROOT_DIR))

# Root directory of the dataset
DATA_DIR = os.path.join(ROOT_DIR, "dataset/wad")
print('Data Directory: {}'.format(DATA_DIR))

# Directory to save logs and trained model
LOGS_DIR = os.path.join(ROOT_DIR, "logs")
print('Logs and Model Directory: {}'.format(LOGS_DIR))

# Local path to trained coco weights file
COCO_MODEL_PATH = os.path.join(LOGS_DIR, 'mask_rcnn_coco.h5')
# Download COCO trained weights from Releases if needed
if not os.path.exists(COCO_MODEL_PATH):
    utils.download_trained_weights(COCO_MODEL_PATH)

Project Directory: /home/shared/project/MaskTrack_RCNN
Data Directory: /home/shared/project/MaskTrack_RCNN/dataset/wad
Logs and Model Directory: /home/shared/project/MaskTrack_RCNN/logs


Using TensorFlow backend.


## Configuration

In [2]:
import wad_data # --> made change here
#import wad_data_testing_optimization as wad_data

cfg = wad_data.WADConfig()
cfg.IMAGES_PER_GPU = 3
cfg.BATCH_SIZE = 3
cfg.STEPS_PER_EPOCH = 5
cfg.display()


Configurations:
BACKBONE                       resnet101
BACKBONE_STRIDES               [4, 8, 16, 32, 64]
BATCH_SIZE                     3
BBOX_STD_DEV                   [ 0.1  0.1  0.2  0.2]
COMPUTE_BACKBONE_SHAPE         None
DETECTION_MAX_INSTANCES        100
DETECTION_MIN_CONFIDENCE       0.7
DETECTION_NMS_THRESHOLD        0.3
FPN_CLASSIF_FC_LAYERS_SIZE     1024
GPU_COUNT                      1
GRADIENT_CLIP_NORM             5.0
IMAGES_PER_GPU                 3
IMAGE_MAX_DIM                  1024
IMAGE_META_SIZE                48
IMAGE_MIN_DIM                  800
IMAGE_MIN_SCALE                0
IMAGE_RESIZE_MODE              square
IMAGE_SHAPE                    [1024 1024    3]
LEARNING_MOMENTUM              0.9
LEARNING_RATE                  0.001
LOSS_WEIGHTS                   {'rpn_class_loss': 1.0, 'rpn_bbox_loss': 1.0, 'mrcnn_class_loss': 1.0, 'mrcnn_bbox_loss': 1.0, 'mrcnn_mask_loss': 1.0}
MASK_POOL_SIZE                 14
MASK_SHAPE                     [28, 28]
MAX_GT_I

## Dataset

load_mask has been tested. green light .. green light

In [3]:
dataset_train = wad_data.WADDataset()
dataset_val = dataset_train.load_data(DATA_DIR, "train", val_size=0.2)

dataset_train.prepare()
dataset_val.prepare()

dataset_val.save_data_to_file(os.path.join(LOGS_DIR, "last_run_validation.pkl"))

OPTIMIZATION LOG: __init__ time: 7.462501525878906e-05
/home/shared/project/MaskTrack_RCNN/dataset/wad/train
OPTIMIZATION LOG: __init__ time: 7.176399230957031e-05
OPTIMIZATION LOG: save_data_to_file time: 0.0003447532653808594
OPTIMIZATION LOG: load_data time: 0.004624128341674805
OPTIMIZATION LOG: save_data_to_file time: 0.00034046173095703125


## Training

In [4]:
STARTING_WEIGHTS = os.path.join(LOGS_DIR, 'mask_rcnn_coco.h5')

# Create model in training mode
model = modellib.MaskRCNN(mode="training", config=cfg, model_dir=LOGS_DIR)

model.load_weights(STARTING_WEIGHTS, by_name=True, exclude=[
            "mrcnn_class_logits", "mrcnn_bbox_fc",
            "mrcnn_bbox", "mrcnn_mask"])

model.train(dataset_train, dataset_val, 
            learning_rate=cfg.LEARNING_RATE, 
            epochs=1, 
            layers='heads')

print('----- TRAINED HEADS -----')

model.train(dataset_train, dataset_val, 
            learning_rate=cfg.LEARNING_RATE, 
            epochs=1, 
            layers='all')

print('----- TRAINED ALL -----')

CURRENTLY EDITTING FOR OPTIMIZATION, DO NOT USE IN PRODUCTION USE

Starting at epoch 0. LR=0.001

Checkpoint Path: /home/shared/project/MaskTrack_RCNN/logs/wad20180627T1926/mask_rcnn_wad_{epoch:04d}.h5
Selecting layers to train
fpn_c5p5               (Conv2D)
fpn_c4p4               (Conv2D)
fpn_c3p3               (Conv2D)
fpn_c2p2               (Conv2D)
fpn_p5                 (Conv2D)
fpn_p2                 (Conv2D)
fpn_p3                 (Conv2D)
fpn_p4                 (Conv2D)
In model:  rpn_model
    rpn_conv_shared        (Conv2D)
    rpn_class_raw          (Conv2D)
    rpn_bbox_pred          (Conv2D)
mrcnn_mask_conv1       (TimeDistributed)
mrcnn_mask_bn1         (TimeDistributed)
mrcnn_mask_conv2       (TimeDistributed)
mrcnn_mask_bn2         (TimeDistributed)
mrcnn_class_conv1      (TimeDistributed)
mrcnn_class_bn1        (TimeDistributed)
mrcnn_mask_conv3       (TimeDistributed)
mrcnn_mask_bn3         (TimeDistributed)
mrcnn_class_conv2      (TimeDistributed)
mrcnn_class_bn2   

/home/shared/anaconda3/lib/python3.6/site-packages/tensorflow/python/ops/gradients_impl.py:98: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "
/home/shared/anaconda3/lib/python3.6/site-packages/keras/engine/training.py:2087: UserWarning: Using a generator with `use_multiprocessing=True` and multiple workers may duplicate your data. Please consider using the`keras.utils.Sequence class.
  UserWarning('Using a generator with `use_multiprocessing=True`'


Epoch 1/5
OPTIMIZATION LOG: load_image time: 0.33792567253112793
OPTIMIZATION LOG: load_image time: 0.4164297580718994
OPTIMIZATION LOG: load_image time: 0.3641645908355713
OPTIMIZATION LOG: load_image time: 0.4337754249572754
OPTIMIZATION LOG: load_mask time: 0.6135823726654053
OPTIMIZATION LOG: load_mask time: 0.613203763961792
OPTIMIZATION LOG: load_mask time: 0.5405635833740234
OPTIMIZATION LOG: load_mask time: 0.6875567436218262
OPTIMIZATION LOG: load_image time: 0.2686502933502197
OPTIMIZATION LOG: load_image time: 0.3676314353942871
OPTIMIZATION LOG: load_image time: 0.3950674533843994
OPTIMIZATION LOG: load_image time: 0.38023877143859863
OPTIMIZATION LOG: load_mask time: 0.6698179244995117
OPTIMIZATION LOG: load_mask time: 0.6962757110595703
OPTIMIZATION LOG: load_mask time: 0.72127366065979
OPTIMIZATION LOG: load_mask time: 0.7663042545318604
OPTIMIZATION LOG: load_image time: 0.4296109676361084
OPTIMIZATION LOG: load_image time: 0.4142460823059082
OPTIMIZATION LOG: load_imag

/home/shared/anaconda3/lib/python3.6/site-packages/keras/engine/training.py:2348: UserWarning: Using a generator with `use_multiprocessing=True` and multiple workers may duplicate your data. Please consider using the`keras.utils.Sequence class.
  UserWarning('Using a generator with `use_multiprocessing=True`'


OPTIMIZATION LOG: load_mask time: 0.8890650272369385
OPTIMIZATION LOG: load_image time: 0.4606282711029053
OPTIMIZATION LOG: load_image time: 0.6947495937347412
OPTIMIZATION LOG: load_image time: 0.896533727645874
OPTIMIZATION LOG: load_image time: 0.7138082981109619
OPTIMIZATION LOG: load_image time: 0.9898116588592529
OPTIMIZATION LOG: load_image time: 0.5984041690826416
OPTIMIZATION LOG: load_mask time: 1.434751033782959
OPTIMIZATION LOG: load_mask time: 1.3957135677337646
OPTIMIZATION LOG: load_image time: 0.7203075885772705
OPTIMIZATION LOG: load_mask time: 1.3078081607818604
OPTIMIZATION LOG: load_mask time: 1.3762638568878174
OPTIMIZATION LOG: load_image time: 0.6180577278137207
OPTIMIZATION LOG: load_mask time: 1.2772834300994873
OPTIMIZATION LOG: load_mask time: 1.4534902572631836
OPTIMIZATION LOG: load_mask time: 1.311178207397461
OPTIMIZATION LOG: load_mask time: 1.2791006565093994
OPTIMIZATION LOG: load_image time: 0.7012472152709961
OPTIMIZATION LOG: load_image time: 0.698

...

OPTIMIZATION LOG: load_image time: 0.4198148250579834


  File "/home/shared/anaconda3/lib/python3.6/multiprocessing/connection.py", line 206, in send
    self._send_bytes(_ForkingPickler.dumps(obj))
  File "/home/shared/anaconda3/lib/python3.6/multiprocessing/connection.py", line 404, in _send_bytes
    self._send(header + buf)
  File "/home/shared/anaconda3/lib/python3.6/multiprocessing/connection.py", line 368, in _send
    n = write(self._handle, buf)
BrokenPipeError: [Errno 32] Broken pipe


OPTIMIZATION LOG: load_image time: 0.44556593894958496
OPTIMIZATION LOG: load_image time: 0.6092486381530762
OPTIMIZATION LOG: load_mask time: 0.744929313659668
OPTIMIZATION LOG: load_mask time: 0.7901785373687744
OPTIMIZATION LOG: load_mask time: 0.8748583793640137
OPTIMIZATION LOG: load_image time: 0.34065890312194824
OPTIMIZATION LOG: load_mask time: 0.7916557788848877
OPTIMIZATION LOG: load_image time: 0.3455173969268799
OPTIMIZATION LOG: load_image time: 0.484799861907959
OPTIMIZATION LOG: load_image time: 0.5778903961181641
OPTIMIZATION LOG: load_mask time: 0.8667411804199219
OPTIMIZATION LOG: load_mask time: 0.8134186267852783
OPTIMIZATION LOG: load_mask time: 1.0250041484832764
OPTIMIZATION LOG: load_image time: 0.3287534713745117
OPTIMIZATION LOG: load_mask time: 1.0377733707427979
OPTIMIZATION LOG: load_image time: 0.21868085861206055
OPTIMIZATION LOG: load_image time: 0.5231640338897705
OPTIMIZATION LOG: load_mask time: 0.5272059440612793
OPTIMIZATION LOG: load_image time: 0

OPTIMIZATION LOG: load_image time: 0.2968935966491699
OPTIMIZATION LOG: load_mask time: 1.2300488948822021
OPTIMIZATION LOG: load_mask time: 0.8155872821807861
OPTIMIZATION LOG: load_mask time: 1.2684273719787598
OPTIMIZATION LOG: load_image time: 0.502840518951416
OPTIMIZATION LOG: load_mask time: 0.5670342445373535
OPTIMIZATION LOG: load_image time: 0.3567013740539551
OPTIMIZATION LOG: load_image time: 0.2768564224243164
OPTIMIZATION LOG: load_mask time: 0.708043098449707
OPTIMIZATION LOG: load_image time: 0.4774613380432129
OPTIMIZATION LOG: load_mask time: 0.8742926120758057
OPTIMIZATION LOG: load_image time: 0.3179900646209717
OPTIMIZATION LOG: load_mask time: 0.6562519073486328
OPTIMIZATION LOG: load_mask time: 0.7812843322753906
OPTIMIZATION LOG: load_image time: 0.26191091537475586
OPTIMIZATION LOG: load_image time: 0.32636022567749023
OPTIMIZATION LOG: load_mask time: 0.7154207229614258
OPTIMIZATION LOG: load_image time: 0.4648129940032959
OPTIMIZATION LOG: load_mask time: 0.7

BrokenPipeError: [Errno 32] Broken pipe